In [2]:
# Versão usando GridSerach sobre base de validação, 
# em seguida fazendo uma validação cruzada sobre o restante dos dados.

# Importa bibliotecas necessárias 
import numpy as np
from sklearn.svm import SVC
from sklearn import  model_selection
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn import preprocessing
import pandas as pd

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

# carregando a base de dados
data = pd.read_csv("heart_failure_clinical_records_dataset.csv")
#print(data)

# recupera os valores e nomes de atributos a partir do dataframe
val=data.values
att=data.columns

# preparando os arrays X (atributos de entrada) e y (rótulos)
X = val[:,0:12]
y = val[:,12]
print(X.shape)
print(y.shape)

# nomalização dos dados
#scaler = preprocessing.StandardScaler().fit(X)
#X = scaler.transform(X)

# nomalização dos dados
scaler = preprocessing.MinMaxScaler().fit(X)
X = scaler.transform(X)

#X = preprocessing.normalize(X, norm='l1', axis=0)
print(X)

# Treina o classificador
#Definição dos parâmetros a serem avaliados no ajuste fino do SVM
parameters = [
  {'C': [1, 5, 10, 50, 100, 200], 'kernel': ['linear']},
  {'C': [1, 5, 10, 50, 100, 200], 'kernel': ['poly']},
  {'C': [1, 5, 10, 50, 100, 150, 500], 'gamma': [0.1, 0.01, 0.001, 0.0001, 'scale'], 'kernel': ['rbf']},
]

# separando uma parte para base de validação (20%)
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, random_state=42, stratify=y)


clf = SVC(probability=True)

# GridSearch para customizar os parâmetros sobre base de validação
gs = GridSearchCV(clf, parameters, scoring = 'accuracy', cv=5, n_jobs=5)
gs.fit(X_val, y_val)

from tabulate import tabulate
import pandas as pd
df=gs.cv_results_
print(tabulate(df, headers='keys', tablefmt='psql'))

print(gs.best_params_)

# validação cruzada 10-folds
folds=10
clf=gs.best_estimator_
result = model_selection.cross_val_score(clf, X_train, y_train, cv=folds, n_jobs=5)
print("\nCross Validation Results %d folds:" % folds)
print("Mean accuracy: %.5f" % result.mean())
print("Std: %.5f" % result.std())

# calculando a saída para cada instância de teste
output_prediction = model_selection.cross_val_predict(clf, X_train, y_train, cv=folds, n_jobs=5)
# calculando a matriz de confusão
cm=confusion_matrix(y_train, output_prediction)
print("Confusion Matrix:")
print(cm)




(299, 12)
(299,)
[[0.63636364 0.         0.07131921 ... 1.         0.         0.        ]
 [0.27272727 0.         1.         ... 1.         0.         0.00711744]
 [0.45454545 0.         0.01569278 ... 1.         1.         0.01067616]
 ...
 [0.09090909 0.         0.25988773 ... 0.         0.         0.97508897]
 [0.09090909 0.         0.30492473 ... 1.         1.         0.98220641]
 [0.18181818 0.         0.02207196 ... 1.         1.         1.        ]]


/usr/local/lib/python3.7/dist-packages/tabulate.py:595: UserWarning: Warning: converting a masked element to nan.
  conv(string)


+-----------------+----------------+-------------------+------------------+-----------+----------------+---------------+-----------------------------------------------+---------------------+---------------------+---------------------+---------------------+---------------------+-------------------+------------------+-------------------+
|   mean_fit_time |   std_fit_time |   mean_score_time |   std_score_time |   param_C | param_kernel   | param_gamma   | params                                        |   split0_test_score |   split1_test_score |   split2_test_score |   split3_test_score |   split4_test_score |   mean_test_score |   std_test_score |   rank_test_score |
|-----------------+----------------+-------------------+------------------+-----------+----------------+---------------+-----------------------------------------------+---------------------+---------------------+---------------------+---------------------+---------------------+-------------------+------------------+-------